# 作業 
實作模糊與邊緣檢測

透過 Gaussian Filter 實作模糊操作  
透過 Sobel Filter 實作邊緣檢測

In [14]:
import cv2
import numpy as np

img = cv2.imread('lena.png')

**<font size=5>邊緣檢測</font>**  
**<font size=4>比較 Sobel 如果在 uint8 的情況下做會 overflow 的狀況</font>**

In [15]:
# 轉為灰階圖片
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 對 x 方向以包含負數的資料格式 (cv2.CV_16S) 進行 Sobel 邊緣檢測
img_sobel_x = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=0, ksize=3)   

# 對 x 方向依照比例縮放到所有數值都是非負整數
img_sobel_x = cv2.convertScaleAbs(img_sobel_x)  # 處理變為非負整數

# 對 x 方向直接以非負整數的資料格式 (uint8) 進行 Sobel 邊緣檢測
img_sobel_x_uint8 = cv2.Sobel(img_grey, cv2.CV_8U, dx=1, dy=0, ksize=3)

# 組合 + 顯示圖片
img_show = np.hstack((img_grey, img_sobel_x, img_sobel_x_uint8))
while True:
    # 比較 Sobel 邊緣檢測的過程中針對負數操作的不同產生的差異
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break


**<font size=4>比較一次與兩次計算偏微分的結果</font>**

In [18]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 求一次導數取得邊緣檢測結果
img_sobel_x = cv2.Sobel(img_grey, cv2.CV_16S, dx=1, dy=0, ksize=3)   
img_sobel_x = cv2.convertScaleAbs(img_sobel_x)  

# 求二次導數取得邊緣檢測結果
img_sobel_x2 = cv2.Sobel(img_grey, cv2.CV_16S, dx=2, dy=0, ksize=3)
img_sobel_x2 = cv2.convertScaleAbs(img_sobel_x2)

# 組合 + 顯示圖片
img_show = np.hstack((img_grey, img_sobel_x, img_sobel_x2))
while True:
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

# 範例
實作模糊與邊緣檢測

透過 Gaussian Filter 實作模糊操作  
透過 Sobel Filter 實作邊緣檢測

In [7]:
import cv2
import numpy as np

img = cv2.imread('lena.png')

**<font size=5>模糊</font>**

In [12]:
img_blur3 = img.copy()
img_blur5 = img.copy()
img_blur7 = img.copy()

# 重複多次 Gaussian 模糊的操作來加深模糊的程度
img_blur3 = cv2.GaussianBlur(img_blur3, (3, 3), 0)  #0是讓函數根據核函數的大小自行計算高斯函數沿著 X 或 Y 方向的標準差
img_blur3 = cv2.GaussianBlur(img_blur3, (3, 3), 0)  #標準差可以分別調整 X 和 Y的，若只指定一個，則 X 和 Y 的設定會一樣
img_blur3 = cv2.GaussianBlur(img_blur3, (3, 3), 0)

# 重複多次 Gaussian 模糊的操作來加深模糊的程度
img_blur5 = cv2.GaussianBlur(img_blur5, (5, 5), 0)
img_blur5 = cv2.GaussianBlur(img_blur5, (5, 5), 0)
img_blur5 = cv2.GaussianBlur(img_blur5, (5, 5), 0)

# 重複多次 Gaussian 模糊的操作來加深模糊的程度
img_blur7 = cv2.GaussianBlur(img_blur7, (7, 7), 0)
img_blur7 = cv2.GaussianBlur(img_blur7, (7, 7), 0)
img_blur7 = cv2.GaussianBlur(img_blur7, (7, 7), 0)

# 組合 + 顯示圖片
img_show = np.hstack((img, img_blur3, img_blur5, img_blur7))
while True:
    cv2.imshow('Gaussian Blur', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

→由上可見，在重複 Gaussian 模糊次數相同下，filter越大，模糊程度越大

**<font size=5>邊緣檢測</font>**  

**<font size=4>組合x-axis, y-axis</font>**

In [13]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 對 x 方向做 Sobel 邊緣檢測
img_sobel_x = cv2.Sobel(img_grey, 
                        cv2.CV_16S, # Sobel函數求完導數後會有負值，還會有大於255的值。但原圖像是uint8，即8位無符號數，
                                    # 所以Sobel建立的圖像位數不夠，位數會被截斷。因此要先轉換成16位有符號的數據類型:int16
                        dx=1, dy=0, # 對x和y求導數的次數
                        ksize=3)    # ksize是Sobel算子的大小，必須為1、3、5、7
img_sobel_x = cv2.convertScaleAbs(img_sobel_x)  # 處理變為非負整數

# 對 y 方向做 Sobel邊緣檢測
img_sobel_y = cv2.Sobel(img_grey, cv2.CV_16S, dx=0, dy=1, ksize=3)    
img_sobel_y = cv2.convertScaleAbs(img_sobel_y)  

# x, y方向的邊緣檢測後的圖各以一半的權重進行合成
img_sobel_combine = cv2.addWeighted(img_sobel_x, 0.5, img_sobel_y, 0.5, 0)

# 組合 + 顯示圖片
img_show = np.hstack((img_sobel_x, img_sobel_y, img_sobel_combine))
while True:
    cv2.imshow('Edge Detection', img_show)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break